# Projet 1, TP2 - Descente de gradient déterministe

Pour l'exemple de régression linéaire du TP précédent, les méthodes de résolution directe que vous avez déjà utilisées calculent une matrice $2 \times 2$ et l'inversent. 
Dans la suite, on va faire comme si cette matrice et cette résolution de système étaient trop coûteuses pour notre petit ordinateur. 
À la place, on va considérer une méthode itérative qui ne considère pas de matrice : **la descente de gradient**.

*Remarque :* Si vous souhaitez lire des articles, le domaine de l'optimisation dénote généralement $x$ la variable selon laquelle on optimise, et $x \mapsto f(x)$ la fonction coût. En apprentissage automatique, $x$ dénote généralement les *inputs* et $f$ est le modèle qu'on évalue, de paramètres dénotés $\theta$. On minimise une fonction coût $\theta \mapsto \mathcal{L}(\theta)$ par rapport aux paramètres $\theta$. Il faut faire attention à ne pas mélanger les notations en fonction de ce que l'on lit. Ici on va utiliser le formalisme de l'apprentissage.

In [ ]:
import numpy as np
from numpy.polynomial.polynomial import Polynomial

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import torch

rng_seed = sum(ord(c) ** 2 for c in "R5.A.12-ModMath")
torch.manual_seed(rng_seed)

### La fonction à minimiser

Ci-dessous, on définit et on affiche la fonction $\theta \mapsto \mathcal{L}(\theta)$ que l'on cherche à minimiser, définie par
$$ \mathcal{L}(\theta) = P(\theta) / (1 + \theta^2) , $$
avec $P$ un polynôme de degré 4 bien choisi. 
Elle admet deux minima locaux, dont un minimum global. 
En ces points, la fonction, que l'on appelle *loss* vaut `-25.33866607` ou `-8.34234203`. 
L'objectif de cette partie est de trouver numériquement ces minima et les *loss* associées.

In [ ]:
# Définition et affichage de la fonction à minimiser

# coefficients polynomiaux bien choisis pour l'exemple 😇
example_coeffs = (
    16.8350101057134,
    10.621467774429892,
    -45.29852223580801,
    3.380171476274114,
    6.068075889020002,
)
example_poly = Polynomial(example_coeffs)


def example_loss_fun(param):
    return example_poly(param) / (1.0 + param**2)


param_min, param_max = -4.5, 3.5
param_vals_continuous = np.linspace(param_min, param_max, 500)
loss_vals_continuous = example_loss_fun(param_vals_continuous)

trace_continuous = go.Scatter(
    x=param_vals_continuous, y=loss_vals_continuous, name="erreur"
)
fig_continuous = go.Figure(
    data=[trace_continuous],
    layout=dict(
        width=500,
        height=250,
        margin=dict(l=20, r=20, b=20, t=20),
        xaxis_title="param",
        yaxis_title="erreur",
    ),
)
fig_continuous.show()

### Approximation linéaire

Pour différents points $\theta_0$, on trace la droite tangente d'équation
$$ y = \ell(\theta; \theta_0) := \nabla\mathcal{L}(\theta_0) (\theta - \theta_0) + \mathcal{L}(\theta) . $$
où $\nabla\mathcal{L}$ dénote la dérivée de $\mathcal{L}$. Pour les fonctions à plusieurs variables, on appellera cela le gradient, que l'on définira au prochain TP.

Zoomer autour de ces points pour vérifier qu'on a bien $\ell(\theta, \theta_0) \approx \mathcal{L}(\theta)$. 

In [ ]:
# Définition de la fonction dérivée et affichage de l'approximation linéaire en quelques points

example_poly_deriv = example_poly.deriv()


def example_loss_fun_deriv(param):
    poly_vals = example_poly(param)
    dpoly_vals = example_poly_deriv(param)
    param_sq_p1 = 1.0 + param**2
    return (dpoly_vals * param_sq_p1 - 2.0 * param * poly_vals) / param_sq_p1**2


fig = go.Figure(fig_continuous)

param_vals = np.array([-4.0, -2.5, -0.5, 1.0, 2.5])
loss_vals = example_loss_fun(param_vals)
dloss_vals = example_loss_fun_deriv(param_vals)

fig.add_scatter(
    x=param_vals,
    y=loss_vals,
    mode="markers+text",
    marker=dict(color="red"),
    text=["(a)", "(b)", "(c)", "(d)", "(e)"],
    textposition="top center",
)

for theta_0, loss_0, dloss_0 in zip(param_vals, loss_vals, dloss_vals):
    theta_span = theta_0 + 0.7 * np.array([-1.0, 1.0])
    affine_span = dloss_0 * (theta_span - theta_0) + loss_0
    fig.add_scatter(
        x=theta_span, y=affine_span, mode="lines", line=dict(color="red", dash="dot")
    )

fig.update_yaxes()
y_min_tmp, y_max_tmp = loss_vals_continuous.min(), loss_vals_continuous.max()
y_min = y_min_tmp - 0.1 * (y_max_tmp - y_min_tmp)
y_max = y_max_tmp + 0.1 * (y_max_tmp - y_min_tmp)
fig.update_layout(
    xaxis=dict(title="param", range=[param_min, param_max]),
    yaxis=dict(title="erreur", range=[y_min, y_max]),
    showlegend=False,
)
fig.show()

## Partie 1 - L'algorithme de descente de gradient

À partir d'un paramètre initial $\theta_0$, on cherche à se rapprocher du puits le plus proche. 
Pour cela, on définit un nouveau point $\theta_1$ qui va dépendre notamment du signe de la dérivée : 
- Si $\nabla\mathcal{L}(\theta_0)$ est strictement positive (comme au point $(c)$ ci-dessus), alors la fonction est croissante, et on veut choisir $\theta_1 < \theta_0$. 
- si $\nabla\mathcal{L}(\theta_0) < 0$, alors $\theta_1 > \theta_0$. Enfin, si $\mathcal{L}(\theta_0) = 0$. 
- si $\nabla\mathcal{L}(\theta_0) = 0$, on supposera qu'on est sur un minimum.

Cela revient à «glisser» le long de la tangente, définir un nouveau paramètre $\theta_1$ correspondant à une *loss* plus faible, puis recommencer pour définir un $\theta_2$, etc.
On définit ainsi une suite $(\theta_t)_{t \geq 0}$ où l'erreur s'améliore petit à petit.
Ci-dessous, on fournit une fonction `plot_error_evolution` qui permet d'afficher la suite et l'évolution de l'erreur en fonction de $t$.

In [4]:
def plot_error_evolution(theta_t, fig=None, min_val=None):
    alpha = np.linspace(0.0, 1.0, len(theta_t))
    loss_t = example_loss_fun(theta_t)

    if fig is None:
        fig = make_subplots(rows=1, cols=2)
        fig.add_trace(trace_continuous, row=1, col=1)
        fig.update_layout(
            width=800, height=250, margin=dict(l=20, r=20, b=20, t=20), showlegend=False
        )
        fig.update_yaxes(type="log", row=1, col=2)

    params = dict(mode="markers", marker=dict(color=alpha), row=1, col=1)
    fig.add_scatter(x=theta_t, y=loss_t, name="(a)", **params)

    if min_val is None:
        min_val = loss_t.min() + 1e-8
    fig.add_scatter(y=loss_t - min_val, row=1, col=2)

    return fig

### 1.1. Descente brutale vers le puits

L'idée la plus simple est de faire un pas dans la bonne direction, 
$$ \theta_{t+1} = \theta_t - \gamma\ {\rm signe}\bigl( \nabla\mathcal{L}(\theta_t) \bigr) . $$
Le paramètre $\gamma$ s'appelle le *learning rate*. 
On obtient une approximation $\mathcal{L}(\theta_{t+1}) \approx \mathcal{L}(\theta_t) - \gamma | \nabla\mathcal{L}(\theta_t) |$, ce qui indique que l'erreur devrait décroitre.

Pour $\theta_0 = -3.8$, calculer les points $\theta_t$ pour $t$ allant de $1$ à $10$ avec $\gamma = 0.5$. À l'aide de la fonction `plot_error_evolution`, afficher ces points ainsi que l'évolution de la différence entre $\mathcal{L}(\theta_t)$ et le minimum le plus proche en fonction de $t$. Est-ce qu'on atteint bien le minimum le plus proche ? Qu'en est-il si on démarre avec $\theta_0 = 3.3$ ?

In [ ]:
gamma = 0.5
num_iters = 10
theta_t = np.repeat([[-3.8, 3.3]], num_iters, axis=0)
for t in range(1, num_iters):
    #TODO appliquer les itérations
    # theta_t[t] = ...

fig = plot_error_evolution(theta_t[:, 0], min_val=-25.33866607)
fig = plot_error_evolution(theta_t[:, 1], fig=fig, min_val=-8.34234203)
fig.show()

### 1.2. La descente de gradient classique

Plutôt que de faire évoluer le paramètre avec un pas fixe, on choisit le pas en fonction de la proximité au puits :
$$ \theta_{t+1} = \theta_t - \gamma\ \nabla\mathcal{L}(\theta_t) , $$
ce qui correspond à une approximation $\mathcal{L}(\theta_{t+1}) \approx \mathcal{L}(\theta_t) - \gamma \bigl( \mathcal{L}(\theta_t) \bigr)^2$.

Pour $\theta_0 = -3.8$, calculer les points $\theta_t$ pour $t$ allant de $1$ à $10$ avec $\gamma = 0.5$. À l'aide de la fonction `plot_error_evolution`, afficher ces points ainsi que l'évolution de la différence entre $\mathcal{L}(\theta_t)$ et le minimum le plus proche en fonction de $t$. Est-ce qu'on atteint bien le minimum le plus proche ? Qu'en est-il si on démarre avec $\theta_0 = 3.3$ ?

In [ ]:
#TODO appliquer l'algorithme de descente de gradient et afficher les résultats

### 1.3. L'effet du *learning rate*

Effectuer la même étude de la descente de gradient avec $\gamma = 10^-3$ et $120$ itérations. Commenter.

*Remarque :* Le *learning rate* $\gamma$ est un **hyperparamètre**, c'est-à-dire un paramètre qu'on n'apprend pas. Il est généralement entre $10^{-2}$ et $10^{-5}$ en fonction des applications. Voici une brève [vidéo d'explication](https://www.youtube.com/watch?v=TwJ8aSZoh2U) (en anglais) sur le comportement de l'apprentissage en fonction de cet hyperparamètre.

In [ ]:
#TODO

## Partie 2 - Différentiation automatique

### 2.1. Le graphe de computation

Un intérêt majeur de PyTorch est que les tenseurs ont une «mémoire» des différentes opérations dont ils sont issus à travers un *graphe de calcul*, ce qui permet de calculer automatiquement la dérivée. Il y a pour ça deux approches, à partir d'une assignation `y = f(x0, x1)` :
1. la fonction `torch.autograd.grad(y, (x0, x1))` qui renvoie un tuple `(dy/dx0, dy/dx1)` ;
2. la méthode `torch.Tensor.backward()` qui ajoute `dy/dx0` à `x0.grad`, et de même pour `x1`.

Pour appliquer ces approches, il faut **au préalable** indiquer que les tenseurs `x0` et `x1` sont différentiables, soit avec l'argument `requires_grad=True` lors de la construction, soit avec la méthode `x0.requires_grad_()`.

In [ ]:
# deux constructions équivalentes
x0 = torch.randn((), requires_grad=True)
x1 = torch.randn(()).requires_grad_()

y0 = x0 * x1
grad0, grad1 = torch.autograd.grad(y0, (x0, x1))
y1 = x0 * x1  # même calcul
y1.backward()

# comparaison des méthodes
x0.grad - grad0, x1.grad - grad1

### 2.2. Quelques avertissements

*Avertissements :* 
1. Par défaut, le graphe de calcul est supprimé après l'appel de `grad` ou de `backward`. On ne peut pas effectuer cette opération deux fois sur le même tenseur ;
2. Avec `grad(y, x0)`, la sortie est un 1-tuple `(dy/dx0,)` ;
3. Avec `backward`, on **ajoute** le gradient à `x.grad`, il faut penser à le remettre à zéro lorsque nécessaire ;
4. Si on ne fait pas attention et qu'on enchaîne trop d'opérations avec un tenseur différentiable, l'impact mémoire du graphe de calcul risque d'exploser.

On illustre le troisième point ci-dessous.

In [ ]:
# deux constructions équivalentes
x0 = torch.randn((), requires_grad=True)
x1 = torch.randn(()).requires_grad_()

y = x0 * x1
z = x0 + x1

y.backward()
z.backward()
somme_grads = x1.item() + 1.0  # la méthode item fait la conversion tenseur -> float

print(x0.grad.item() - somme_grads)

### 2.3. Modification d'un tenseur différentiable

Pour modifier un tenseur différentiable, il faut faire l'opération dans un environnement `torch.no_grad()`.

*Remarque :* Avec TensorFlow, la philosophie est opposée : toutes les opérations sont `no_grad` par défaut et il faut spécifier lorsque l'on souhaite grapher les calculs.

Observer cette contrainte dans la cellule suivante.

In [41]:
def naive_update(x):
    x -= 0.1
    return x

def no_grad_update(x):
    with torch.no_grad():
        x -= 0.1
    return x

x = torch.randn((), requires_grad=True)

#TODO comparer les deux fonctions

### 2.4. Application au problème d'optimisation

En utilisant `torch.autograd.grad`, mettre en place l'algorithme de descente de gradient avec $\theta_0 = -3.8$.
Penser à utiliser `Tensor.item()` pour sauvegarder les valeurs de $\theta_t$ sans conserver le graphe de calcul.

Au prochain TP, nous utiliserons la structure `Optimizer` de PyTorch qui permet d'utiliser `backwards` sans avoir de souci avec l'accumulation des gradients, et sans avoir à se soucier des environnements `no_grad`.

In [ ]:
#TODO

## Partie 3 - Pour aller plus loin

On présente ici une amélioration possible à la descente de gradient, et des liens avec le contexte de l'apprentissage automatique.

### 3.1. Descente avec moment

Une amélioration possible de la descente de gradient est l'ajout d'un moment, c'est-à-dire une inertie à l'amplitude de descente. 
L'algorithme implique maintenant une nouvelle variable $g_t$ représentant le gradient intertiel, et s'écrit
$$ g_{t+1} = \nabla\mathcal{L}(\theta_t) + \mu g_t, \qquad \theta_{t+1} = \theta_t - g_{t+1} . $$
pour une paramètre $\mu$ appelé le *momentum*.
Voici une [vidéo d'explication](https://www.youtube.com/watch?v=r-rYz_PEWC8) (en anglais). 
Cela permet parfois de sortir de minima locaux.

*Remarque :* Il existe encore d'autres méthodes, qui choisisse par exemple $\gamma$ en fonction du comportement de la fonction, ou qui se basent sur des analyses plus fine comme avec la méthode de Newton ou les méthodes quasi-Newton. Pour le deep learning, on observera au prochain TP le comportement de la méthode [Adam](https://arxiv.org/abs/1412.6980).

Mettre en place cette méthode avec $\gamma = 10^{-2}$, $\mu = 0.95$ et $\theta_0 = 3.3$, sur $200$ itérations. Afficher les résultats.

In [ ]:
#TODO

### 3.2. L'intérêt des *checkpoints*

Recommencer avec $\gamma = 10^{-2}$, $\mu = 0.97$ et $\theta_0 = -3.8$, sur $200$ itérations.

On voit que l'erreur diminue, puis augmente fortement. Lors d'un apprentissage, il est intéressant de sauvegarder le meilleur résultat au cas où on sortirait du «bon» puits. On peut alors recommencer ponctuellement depuis ce point avec un *learning rate* plus faible.

In [ ]:
#TODO

### 3.3. Entraîner avec divers paramètres initiaux

On voit dans l'exemple précédent que selon où le paramètre initial se situe, la descente de gradient ne converge pas vers le même minimum. 
En l'occurrence, il y a deux puits, et on converge vers l'un ou l'autre en fonction qu'on choisisse $\theta_0$ à gauche ou à droite du maximum local central.
Ci-dessous, un exemple de fonction bien plus pathologique (fonction de Whitley, issue de [ce rapport, Sec. A.7](https://www.uni-goettingen.de/de/document/download/9aa76b19d6bc767fb1f9733b21854cb5.pdf/Bachelorarbeit_Brachem_Carsten.pdf)), plus similaire à ce qui apparaît en apprentissage.
On voit bien qu'il peut y avoir de très diverses valeurs de *loss* finale. 
Ainsi, en *deep learning*, il faut parfois entrainer plusieurs modèles initialisés différemment, et choisir le meilleur.

Utiliser `torch.randn` pour initialiser plusieurs $\theta_0$, et effectuer plusieurs descentes de gradient en même temps. Sélectionner ensuite le meilleur paramètre. On pourra utiliser une descente de gradient standard ou une descente plus élaborée.

In [ ]:
def whitney_loss_fun(x):
    xs = (-1.6681, 0.0369, -2.0580, -1.4312, -1.9171, -0.5520)
    x_m1 = (1.0 - x) ** 2
    tot = x_m1**2 / 4000.0 - torch.cos(x_m1**2) + 1.0
    for xi in xs:
        ener_i = ((xi - x) ** 2 + x_m1) ** 2
        tot += ener_i / 4000.0 - torch.cos(ener_i) + 1.0
    return tot


x = torch.linspace(-1.0, 1.0, 2000)
y = whitney_loss_fun(x)
go.Figure(
    data=[go.Scatter(x=x.numpy(), y=y.numpy())],
    layout=dict(
        width=800,
        height=250,
        margin=dict(l=20, r=20, b=20, t=20),
        xaxis_title="param",
        yaxis_title="erreur",
    ),
)